In [4]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sb
import torch
import torch.nn as nn
import numpy as np
import plotly.express as px
import plotly.graph_objects as go  
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [5]:
data = yf.download('AAPL', start='2018-01-01', end='2025-06-01')

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [6]:
print(data.head())

Price           Close       High        Low       Open     Volume
Ticker           AAPL       AAPL       AAPL       AAPL       AAPL
Date                                                             
2018-01-02  40.426823  40.436212  39.722768  39.933986  102223600
2018-01-03  40.419788  40.964259  40.356426  40.490195  118071600
2018-01-04  40.607533  40.710794  40.384583  40.492536   89738400
2018-01-05  41.069870  41.156702  40.612235  40.703762   94640000
2018-01-08  40.917328  41.213030  40.818757  40.917328   82271200


In [7]:
df = data[['Open', 'High', 'Low', 'Close', 'Volume']].copy()

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1863 entries, 2018-01-02 to 2025-05-30
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   (Open, AAPL)    1863 non-null   float64
 1   (High, AAPL)    1863 non-null   float64
 2   (Low, AAPL)     1863 non-null   float64
 3   (Close, AAPL)   1863 non-null   float64
 4   (Volume, AAPL)  1863 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 87.3 KB


In [9]:
df.isnull().sum()

Price   Ticker
Open    AAPL      0
High    AAPL      0
Low     AAPL      0
Close   AAPL      0
Volume  AAPL      0
dtype: int64

In [10]:
df.describe()

Price,Open,High,Low,Close,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
count,1863.000000,1863.000000,1863.000000,1863.000000,1.863000e+03
mean,127.300563,128.735445,125.980373,127.432248,9.787064e+07
std,61.763485,62.392101,61.183128,61.837595,5.484645e+07
min,34.297222,34.711705,33.825570,33.870831,2.323470e+07
25%,59.622277,60.242856,58.158028,59.401503,6.003700e+07
50%,138.394745,140.073755,136.009659,138.299805,8.449680e+07
75%,172.565178,174.630477,171.412957,172.540756,1.188404e+08
max,257.568678,259.474086,257.010028,258.396667,4.265100e+08


In [11]:
df.shape

(1863, 5)

In [12]:
def computeRSI(data, periods=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=periods).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=periods).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

In [13]:
df['RSI'] = computeRSI(df['Close'])

In [14]:
df = df.dropna()

# Data Preprocessing

In [15]:
scaler = MinMaxScaler()
dfScaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
closeScaler = MinMaxScaler()
closeScaler.fit(df[['Close']])

MinMaxScaler()

# create sequences e.g., 60-day windows

In [16]:
def createSequences(data, seqLength=60):
    X, y = [], []
    for i in range(len(data) - seqLength):
        X.append(data[i:i + seqLength, [0, 1, 2, 4, 5]])
        #X.append(data[i:i + seqLength]) # Shape: [seqLenght, 5]
        y.append(data[i + seqLength, 3]) # Index 3 is Close
    
    return np.array(X), np.array(y)

In [17]:
#features = dfScaled[['Open', 'High', 'Low', 'Close', 'Volume']].values
features = dfScaled[['Open', 'High', 'Low', 'Close', 'Volume', 'RSI']].values

X, y = createSequences(features)
print(f"X shape: {X.shape}, y shape: {y.shape}")

X shape: (1790, 60, 5), y shape: (1790,)


# Create train test splits

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to tensors

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [20]:
X_trainTensor = torch.tensor(X_train, dtype=torch.float32).to(device)
X_testTensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_trainTensor = torch.tensor(y_train, dtype=torch.float32).to(device)
y_testTensor = torch.tensor(y_test, dtype=torch.float32).to(device)
print("X_trainTensor shape:", X_trainTensor.shape)  # Debug


X_trainTensor shape: torch.Size([1432, 60, 5])


# Create Model

## Using LSTM (Long Short-Term Memory) 
Reasons for using LSTM is thats its a type of recurrent neural network (RNN). That is effective at handling time series data.

In [21]:
class StockLSTM(nn.Module):
    def __init__(self, inputSize=5, hiddenUnits=50, numLayers=2, dropout=0.2):
        super(StockLSTM, self).__init__()
        self.lstm = nn.LSTM(inputSize, hiddenUnits, numLayers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hiddenUnits, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

# Training Loop Function

In [22]:
def train(model, X_train, y_train, X_test, y_test, loss_fn, epochs, optimizer, device):
    model.to(device)
    trainDataset = TensorDataset(X_train, y_train)
    trainLoader = DataLoader(trainDataset, batch_size=32, shuffle=True)

    trainLosses = []
    for epoch in tqdm(range(epochs)):
        model.train()
        epochLoss = 0
        for X_batch, y_batch in trainLoader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch.unsqueeze(1)) # Make sure y_batch shape matches
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epochLoss += loss.item()
        
        trainLosses.append(epochLoss / len(trainLoader))
        if epoch % 10 == 0:
            print(f'Epoch: {epoch}, Train Loss: {trainLosses[-1]:.4f}')

    return trainLosses

# Train Model

In [28]:
model1 = StockLSTM(inputSize=5, hiddenUnits=50, numLayers=2, dropout=0.2).to(device)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model1.parameters(), lr=0.001)
resultsTrain = train(
                model1, 
                X_trainTensor,
                y_trainTensor,
                X_testTensor, 
                y_testTensor, 
                loss_fn=loss_fn, 
                epochs=100, 
                optimizer=optimizer, 
                device=device
            )

  1%|          | 1/100 [00:00<00:26,  3.69it/s]

Epoch: 0, Train Loss: 0.0508


 11%|█         | 11/100 [00:01<00:08, 10.83it/s]

Epoch: 10, Train Loss: 0.0007


 21%|██        | 21/100 [00:02<00:06, 11.38it/s]

...

 83%|████████▎ | 83/100 [00:07<00:01, 11.78it/s]

Epoch: 80, Train Loss: 0.0003


 91%|█████████ | 91/100 [00:08<00:00, 10.94it/s]

Epoch: 90, Train Loss: 0.0003


100%|██████████| 100/100 [00:08<00:00, 11.16it/s]


# Evaluate Model

In [24]:
def evaluateModel(model, X_test, y_test, scaler, device):
    model.eval()
    with torch.no_grad():
        preds = model(X_test).cpu().numpy()
        y_testNp = y_test.cpu().numpy()
    
    preds = scaler.inverse_transform(preds)
    y_testNp = scaler.inverse_transform(y_testNp.reshape(-1, 1))
    mae = mean_absolute_error(y_testNp, preds)
    rmse = np.sqrt(mean_squared_error(y_testNp, preds))
    print(f'MAE: {mae:.4f}, RMSE: {rmse:.4f}')

    return preds, y_testNp, mae, rmse

In [29]:
predictions, y_testNp, mae, rmse = evaluateModel(model1, X_testTensor, y_testTensor, closeScaler, device)

MAE: 1.9420, RMSE: 2.6339


# Plot Predictions

In [26]:
def plotPredictions(actual, predicted, title='AAPL Close Price Forecasting', savePath='plot'):
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=actual.flatten(), name='Actual Close Prices', line=dict(color='blue')))
    fig.add_trace(go.Scatter(y=predicted.flatten(), name='Predicted Close Prices', line=dict(color='orange')))
    fig.update_layout(
        title=title,
        xaxis_title='Time',
        yaxis_title='Close Price (USD)',
        template='plotly_dark',
        hovermode='x unified'
    )

    fig.show()

In [30]:
plotPredictions(y_testNp, predictions)